In [62]:
import numpy as np
import pickle as pkl
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

from shapely.geometry import box

In [2]:
fin = open('../dataset/danik_bhaskar_png_bbox_maps.pkl', 'rb')
bbox_maps = pkl.load(fin)
fin.close()

In [136]:
new_bbox_maps = {}
for box_key in bbox_maps.keys():
    sample_bbox = bbox_maps[box_key]
    
    title_bboxes = []
    mapped = {}
    for art in sample_bbox:
        art_poly = box(*art)
        tit_no = 0
        for tit in sample_bbox:
            tit_poly = box(*tit)
            if art_poly.contains(tit_poly) and not art_poly.equals(tit_poly):
                title_bboxes.append(tit)
                if tit_no not in mapped:
                    mapped[tit_no] = 1
            tit_no += 1

    article_bboxes = []
    for i in range(len(sample_bbox)):
        if i not in mapped:
            article_bboxes.append(sample_bbox[i])
            
    minn_width = 1000
    minn_height = 1000

    mapped_bbox = {}
    bbox_no = 0
    for bbox in article_bboxes:
        minn_width = min(minn_width, bbox[3]-bbox[1])
        minn_height = min(minn_height, bbox[2]-bbox[0])
        mapped_bbox[bbox_no] = bbox
        bbox_no += 1

    #print minn_width, minn_height

    to_add = min(minn_width, minn_height) 
    #print to_add
    
    adjacency_map = {}
    for k1 in mapped_bbox.keys():
        for k2 in mapped_bbox.keys():
            if k1 != k2:
                bbox1 = [mapped_bbox[k1][0] - to_add, mapped_bbox[k1][1] - to_add, \
                         mapped_bbox[k1][2] + to_add, mapped_bbox[k1][3] + to_add]
                bbox2 = [mapped_bbox[k2][0] - to_add, mapped_bbox[k2][1] - to_add, \
                         mapped_bbox[k2][2] + to_add, mapped_bbox[k2][3] + to_add]
                bbox1_poly = box(*bbox1)
                bbox2_poly = box(*bbox2)

                if bbox1_poly.intersects(bbox2_poly):
                    if k1 not in adjacency_map:
                        adjacency_map[k1] = [k2]
                    else:
                        adjacency_map[k1] += [k2]
                        
    new_bbox_maps[box_key] = {'orignal_bboxes':sample_bbox, 'title_bboxes':title_bboxes, \
                            'article_bboxes':article_bboxes, 'mapped_bboxes':mapped_bbox, 'adjacency_map':adjacency_map}

In [138]:
fout = open('../dataset/danik_bhaskar_png_bbox_maps_adjacency_map.pkl','wb')
pkl.dump(new_bbox_maps, fout)
fout.close()

In [70]:
#2650x4174 --> 330x520

46 290
46


In [133]:
adjacency_map

{0: [6, 8, 11],
 1: [2, 7, 12, 13, 14],
 2: [1, 4, 12, 16],
 3: [4, 5, 6, 15, 16],
 4: [2, 3, 12, 16],
 5: [3, 6],
 6: [0, 3, 5, 11, 15],
 7: [1, 10, 12, 13],
 8: [0, 9, 10, 11, 15, 16],
 9: [8, 10, 12, 16],
 10: [7, 8, 9, 12],
 11: [0, 6, 8, 15],
 12: [1, 2, 4, 7, 9, 10, 16],
 13: [1, 7, 14],
 14: [1, 13],
 15: [3, 6, 8, 11, 16],
 16: [2, 3, 4, 8, 9, 12, 15]}

In [134]:
img = np.zeros((520, 330))
wscale, hscale = 2650/330.0, 4174/520.0

for k in mapped_bbox.keys():
    bb = mapped_bbox[k]
    cv2.rectangle(img, (int(bb[0]/hscale),int(bb[1]/wscale)), (int(bb[2]/hscale),int(bb[3]/wscale)), 255, -1)
    cv2.putText(img,str(k), (int(bb[0]/hscale + 10), int(bb[1]/wscale + 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 0, 1)

cv2.imwrite('test.jpg',img)

True